In [1]:
import networkx as nx
import pandas as pd
import logging
import numpy as np
import random
import os
import sys
import GridCalEngine.api as gce  # For interfacing with the GridCal API
from GridCalEngine.IO.file_handler import FileOpen, FileSave
import simbench as sb
import pandapower as pp

In [2]:

# Add the directory containing utility_script.py to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import GC_utils
import GC_PandaPowerImporter
import GC_DistributionNetworkReconfiguration


In [3]:
print('Algorithms to find the optimal distribution network configuration')

logging.basicConfig(
    level=logging.ERROR,  # Set the log level to DEBUG
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Set the log format
    datefmt='%Y-%m-%d %H:%M:%S'  # Set the date format
)


Algorithms to find the optimal distribution network configuration


In [4]:
#This function call DNR library, executes the selected DNR method and prints the results, to easy the comparison

def executeTest(dnr, method, TieLinesID=[], randomMST=False, one=False, current_power=True, algorithm="prim", NumCandidates=10, fitness_ratio=1, loss_factor=0.02, MutationProbability=0.4, PopulationSize=16, Niter=20, ElitePopulation=2):
    if method == None:
        _, losses = GC_utils.GC_PowerFlow(dnr.grid, config=TieLinesID)
        radiality = GC_utils.CheckRadialConnectedNetwork(dnr.grid)
        print("losses:", losses)
        print("radiality:", radiality)
        print("Loops:")
        for loop in GC_utils.SearchLoopsLines(dnr.grid):
            print(GC_utils.GC_Line_idtag2name_array(dnr.grid,loop))
    else:      
#       try:
        dnr.NumPF=0
        DisabledLinesID = dnr.Solve(method=method, TieLines=TieLinesID, randomMST=randomMST, one=one, current_power=current_power, algorithm=algorithm, NumCandidates=NumCandidates, fitness_ratio=fitness_ratio, loss_factor=loss_factor, MutationProbability=MutationProbability, PopulationSize=PopulationSize, Niter=Niter, ElitePopulation=ElitePopulation)

        _, losses = GC_utils.GC_PowerFlow(dnr.grid, config=DisabledLinesID)
        radiality = GC_utils.CheckRadialConnectedNetwork(dnr.grid)
        print("DNR solution by ", method, ":", GC_utils.GC_Line_idtag2name_array(dnr.grid, DisabledLinesID))
        print("Number of executed Power Flows:", dnr.NumPF)
        print("losses:", losses)
        print("radiality:", radiality)
        print("Loops:")
        for loop in GC_utils.SearchLoopsLines(dnr.grid):
            print(GC_utils.GC_Line_idtag2name_array(dnr.grid,loop))
#        except:
#            print(method, " method failed")        


In [5]:
## load the two networks to test, the 33 and 118 buses cases

In [6]:
caseName = "33 buses case based on Baran and Wu"
gridGC33 = FileOpen("D:\\15_Thesis-code\\02_DistributionNetworkOperationFramework\\03_NetworkExamples\\gridcal\\case33bw.m").open()
TieLinesName118=['line 32','line 118','line 34','line 35','line 36']
TieLinesName33=['21_8_1', '9_15_1', '12_22_1', '18_33_1','25_29_1']
TieLinesID33=GC_utils.GC_Line_Name2idtag_array(gridGC33, TieLinesName33)

In [7]:
caseName = "69 buses"
gridGC69 = FileOpen("D:\\15_Thesis-code\\02_DistributionNetworkOperationFramework\\03_NetworkExamples\\gridcal\\case69.gridcal").open()
TieLinesName69 = ['line 57','line 10','line 69','line 14','line 19']
TieLinesID69=GC_utils.GC_Line_Name2idtag_array(gridGC69, TieLinesName69)

In [8]:
dnr69 = GC_DistributionNetworkReconfiguration.DistributionNetworkReconfiguration(gridGC69, verbose_logging=logging.ERROR)
executeTest(dnr69, 'MSTgreedy', None, randomMST=False, one=False, current_power=True, algorithm="prim")


DNR solution by  MSTgreedy : ['line 8 ', 'line 17', 'line 21', 'line 39', 'line 45']
Number of executed Power Flows: 1
losses: 232802.12136816452
radiality: (True, True, True)
Loops:


In [9]:
GC_utils.CheckRadialConnectedNetwork(gridGC69)

(True, True, True)

In [10]:
[(line, line.bus_from.name, line.bus_to.name) for line in gridGC69.lines]

[(.::0b6ae31555954779810eff35c8cc1336::line 1 , 'Bus 1', 'Bus 2'),
 (.::7631929130c74c308556ba90747aeb29::line 2 , 'Bus 2', 'Bus 3'),
 (.::bec9f770a92b4b9f8c094c4439bcac65::line 3 , 'Bus 3', 'Bus 4'),
 (.::4e77285ff766449cb68ddb3cdfae99a2::line 4 , 'Bus 4', 'Bus 5'),
 (.::33aa4d412a8d4c6c9faa3c7e64848a35::line 5 , 'Bus 5', 'Bus 6'),
 (.::17f134ed0e3b426cb3e869104e29de0c::line 6 , 'Bus 6', 'Bus 7'),
 (.::976f80531008478ca244b1a3475afb90::line 7 , 'Bus 7', 'Bus 8'),
 (.::942080f9d12b48ef98401e8d3ab5fcc9::line 8 , 'Bus 8', 'Bus 9'),
 (.::3614d10c12f144688a4a345633deeeb4::line 9 , 'Bus 9', 'Bus 10'),
 (.::e2dd2262831f441483664b2127cadeb6::line 10, 'Bus 10', 'Bus 11'),
 (.::c18f1ddabc7e42dc9ded7f3038881d24::line 11, 'Bus 11', 'Bus 12'),
 (.::f7687b50ab564fbca8a918a6d594fe7a::line 12, 'Bus 12', 'Bus 13'),
 (.::061f4093eaad4f378d72598a0cca3724::line 13, 'Bus 13', 'Bus 14'),
 (.::ef203c91e34b4aa9894b2a6f3e134ea3::line 14, 'Bus 14', 'Bus 15'),
 (.::0738f69f7fcf441ca4bec1974e147b76::line 15, 'Bu

In [11]:
caseName = "118 buses"
gridGC118 = FileOpen("D:\\15_Thesis-code\\02_DistributionNetworkOperationFramework\\03_NetworkExamples\\gridcal\\case118.m").open()
TieLinesName118 = ['1_2_1', '4_5_1', '3_12_1', '7_12_1', '13_15_1', '14_15_1', '12_16_1', '18_19_1', '19_20_1', '15_19_1', '29_31_1', '27_32_1', '15_33_1', '35_36_1', '39_40_1', '40_41_1', '40_42_1', '34_43_1', '46_47_1', '46_48_1', '47_49_1', '53_54_1', '54_55_1', '54_56_1', '55_56_1', '56_57_1', '56_58_1', '60_61_1', '60_62_1', '61_62_1', '63_64_1', '62_67_1', '65_68_1', '24_70_1', '24_72_1', '70_74_1', '70_75_1', '75_77_1', '78_79_1', '68_81_1', '84_85_1', '85_88_1', '90_91_1', '93_94_1', '82_96_1', '94_96_1', '94_100_1', '95_96_1', '96_97_1', '98_100_1', '99_100_1', '100_101_1', '103_104_1', '104_105_1', '105_106_1', '106_107_1', '108_109_1', '17_113_1', '114_115_1', '76_118_1', '30_17_1', '64_61_1']
TieLinesID118=GC_utils.GC_Line_Name2idtag_array(gridGC118, TieLinesName118)

In [12]:
#Creation of the DNR instances for each grid
dnr33 = GC_DistributionNetworkReconfiguration.DistributionNetworkReconfiguration(gridGC33, verbose_logging=logging.ERROR)
dnr118 = GC_DistributionNetworkReconfiguration.DistributionNetworkReconfiguration(gridGC118, verbose_logging=logging.ERROR)

In [13]:
GC_utils.NetworkReconfiguration(gridGC33, all=True, value_all=True)
print(f"The 33 buses cases is loaded and with all lines connected it has {len(GC_utils.SearchLoopsLines(gridGC33))} loops")
GC_utils.NetworkReconfiguration(gridGC33, all=True, selected_configuration=TieLinesID33, value_all=True, value_configuration=False)
print(f"The 33 buses cases is loaded and with tie lines disconnected it has {len(GC_utils.SearchLoopsLines(gridGC33))} loops")

The 33 buses cases is loaded and with all lines connected it has 5 loops
The 33 buses cases is loaded and with tie lines disconnected it has 0 loops


In [14]:
GC_utils.NetworkReconfiguration(gridGC118, all=True, value_all=True)
print(f"The 118 buses cases is loaded and with all lines connected it has {len(GC_utils.SearchLoopsLines(gridGC118))} loops")
GC_utils.NetworkReconfiguration(gridGC118, all=True, selected_configuration=TieLinesID118, value_all=True, value_configuration=False)
print(f"The 118 buses cases is loaded and with tie lines disconnected it has {len(GC_utils.SearchLoopsLines(gridGC118))} loops")

The 118 buses cases is loaded and with all lines connected it has 62 loops
The 118 buses cases is loaded and with tie lines disconnected it has 0 loops


In [15]:
## executes the power flow and radiality check for the network with disabled tie lines, as it has been configured in the previous step
executeTest(dnr33, None, None)  

losses: 0.2026771169692977
radiality: (True, True, True)
Loops:


In [16]:
## executes the power flow and radiality check for the network with disabled tie lines, as it has been configured in the previous step
executeTest(dnr118, None, None)  

losses: 496.7852924832344
radiality: (True, True, True)
Loops:


In [17]:
GC_utils.NetworkReconfiguration(dnr33.grid, all=True, value_all=True)
_, loss = GC_utils.GC_PowerFlow(gridGC33, config=[])
radiality = GC_utils.CheckRadialConnectedNetwork(gridGC33)
print(f"TieLines: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr33.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )

_, loss = GC_utils.GC_PowerFlow(gridGC33, config=TieLinesID33)
radiality = GC_utils.CheckRadialConnectedNetwork(gridGC33)
print(f"TieLines: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr33.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )


TieLines: The new optimal configuration losses:0.12329082969373946, radiality:(False, True, False), numPF:0 
TieLines: The new optimal configuration losses:0.2026771169692977, radiality:(True, True, True), numPF:0 


In [18]:
GC_utils.NetworkReconfiguration(dnr118.grid, all=True, value_all=True)
_, loss = GC_utils.GC_PowerFlow(gridGC118, config=[])
radiality = GC_utils.CheckRadialConnectedNetwork(gridGC118)
print(f"TieLines: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr118.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )

_, loss = GC_utils.GC_PowerFlow(gridGC118, config=TieLinesID118)
radiality = GC_utils.CheckRadialConnectedNetwork(gridGC118)
print(f"TieLines: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr118.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )


TieLines: The new optimal configuration losses:129.7837122262946, radiality:(False, True, False), numPF:0 
TieLines: The new optimal configuration losses:496.7852924832344, radiality:(True, True, True), numPF:0 


In [19]:
## executes the power flow and radiality check for the original network with a random set of lines deactivated to achieve a radial network
print("33 buses case")
executeTest(dnr33, None, TieLinesID33)  
print("118 buses case")
executeTest(dnr118, None, TieLinesID118)  

33 buses case
losses: 0.2026771169692977
radiality: (True, True, True)
Loops:
118 buses case
losses: 496.7852924832344
radiality: (True, True, True)
Loops:


In [20]:
#Executes the DNR algorithm by using the Merlin method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, 'Merlin', TieLinesID33)  
print("118 buses case")
executeTest(dnr118, 'Merlin', TieLinesID118) 

33 buses case
DNR solution by  Merlin : ['7_8_1', '14_15_1', '28_29_1', '12_22_1', '18_33_1']
Number of executed Power Flows: 20
losses: 0.15237057030557727
radiality: (True, True, True)
Loops:
118 buses case
DNR solution by  Merlin : ['1_2_1', '5_6_1', '4_11_1', '5_11_1', '11_12_1', '3_12_1', '15_17_1', '16_17_1', '17_18_1', '15_19_1', '25_27_1', '26_30_1', '29_31_1', '23_32_1', '35_37_1', '34_37_1', '39_40_1', '41_42_1', '46_47_1', '42_49_1', '45_49_1', '55_56_1', '56_57_1', '56_58_1', '56_59_1', '55_59_1', '60_61_1', '60_62_1', '63_64_1', '38_65_1', '64_65_1', '49_66_1', '49_66_1', '62_66_1', '66_67_1', '47_69_1', '49_69_1', '69_70_1', '71_72_1', '70_75_1', '69_75_1', '77_80_1', '79_80_1', '68_81_1', '89_90_1', '89_90_1', '89_92_1', '89_92_1', '93_94_1', '82_96_1', '94_96_1', '94_100_1', '95_96_1', '96_97_1', '92_102_1', '103_104_1', '105_106_1', '105_108_1', '106_107_1', '17_113_1', '114_115_1', '75_118_1']
Number of executed Power Flows: 119
losses: 5691.864501965072
radiality: (F

In [21]:
#Executes the DNR algorithm by using the Baran method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, 'Baran', TieLinesID33)  
print("118 buses case")
executeTest(dnr118, 'Baran', TieLinesID118) 

33 buses case
DNR solution by  Baran : ['21_8_1', '9_15_1', '7_8_1', '18_33_1', '25_29_1']
Number of executed Power Flows: 8
losses: 0.15652933737440788
radiality: (True, True, True)
Loops:
118 buses case
DNR solution by  Baran : ['1_2_1', '4_5_1', '3_12_1', '7_12_1', '13_15_1', '14_15_1', '12_16_1', '18_19_1', '19_20_1', '15_19_1', '29_31_1', '27_32_1', '15_33_1', '35_36_1', '39_40_1', '40_41_1', '40_42_1', '34_43_1', '46_47_1', '46_48_1', '47_49_1', '53_54_1', '54_55_1', '54_56_1', '55_56_1', '56_57_1', '56_58_1', '60_61_1', '60_62_1', '61_62_1', '63_64_1', '62_67_1', '65_68_1', '24_70_1', '24_72_1', '70_74_1', '70_75_1', '75_77_1', '78_79_1', '68_81_1', '84_85_1', '85_88_1', '90_91_1', '93_94_1', '82_96_1', '94_96_1', '94_100_1', '95_96_1', '96_97_1', '98_100_1', '99_100_1', '100_101_1', '103_104_1', '104_105_1', '105_106_1', '106_107_1', '108_109_1', '17_113_1', '114_115_1', '76_118_1', '30_17_1', '64_61_1']
Number of executed Power Flows: 3
losses: 496.7852924832344
radiality: (Tr

In [22]:
#Executes the DNR algorithm by using the Salkuti method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, 'Salkuti', TieLinesID33)  
print("118 buses case")
executeTest(dnr118, 'Salkuti', TieLinesID118) 

33 buses case
DNR solution by  Salkuti : ['7_8_1', '14_15_1', '11_12_1', '32_33_1', '28_29_1']
Number of executed Power Flows: 10
losses: 0.14163107972388245
radiality: (True, True, True)
Loops:
118 buses case
DNR solution by  Salkuti : ['1_3_1', '4_11_1', '3_5_1', '6_7_1', '11_13_1', '15_17_1', '16_17_1', '19_34_1', '20_21_1', '18_19_1', '28_29_1', '23_32_1', '14_15_1', '35_37_1', '37_39_1', '41_42_1', '37_40_1', '43_44_1', '45_46_1', '46_47_1', '47_69_1', '52_53_1', '55_59_1', '54_56_1', '54_55_1', '56_57_1', '56_58_1', '59_60_1', '60_61_1', '59_61_1', '63_64_1', '62_66_1', '65_68_1', '69_70_1', '71_72_1', '74_75_1', '69_75_1', '70_75_1', '77_78_1', '68_81_1', '83_84_1', '85_89_1', '91_92_1', '92_94_1', '77_82_1', '93_94_1', '94_96_1', '94_95_1', '80_96_1', '92_100_1', '80_99_1', '101_102_1', '100_104_1', '103_105_1', '100_106_1', '105_106_1', '105_108_1', '17_31_1', '27_115_1', '75_118_1', '30_17_1', '64_61_1']
Number of executed Power Flows: 143
losses: 613.2675333259904
radiality:

In [23]:
#Executes the DNR algorithm by using the MSTgreedy method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, 'MSTgreedy', TieLinesID33, randomMST=False, one=False, current_power=True, algorithm="prim")
print("118 buses case")
executeTest(dnr118, 'MSTgreedy', TieLinesID118, randomMST=False, one=False, current_power=True, algorithm="prim") 


33 buses case
DNR solution by  MSTgreedy : ['6_7_1', '10_11_1', '14_15_1', '6_26_1', '32_33_1']
Number of executed Power Flows: 1
losses: 0.16216538274100445
radiality: (True, True, True)
Loops:
118 buses case
DNR solution by  MSTgreedy : ['1_2_1', '4_5_1', '3_12_1', '7_12_1', '13_15_1', '14_15_1', '12_16_1', '18_19_1', '19_20_1', '15_19_1', '29_31_1', '27_32_1', '15_33_1', '35_36_1', '39_40_1', '40_41_1', '40_42_1', '34_43_1', '46_47_1', '46_48_1', '47_49_1', '53_54_1', '54_55_1', '54_56_1', '55_56_1', '56_57_1', '56_58_1', '60_61_1', '60_62_1', '61_62_1', '63_64_1', '62_67_1', '65_68_1', '24_70_1', '24_72_1', '70_74_1', '70_75_1', '75_77_1', '78_79_1', '68_81_1', '84_85_1', '85_88_1', '90_91_1', '93_94_1', '82_96_1', '94_96_1', '94_100_1', '95_96_1', '96_97_1', '98_100_1', '99_100_1', '100_101_1', '103_104_1', '104_105_1', '105_106_1', '106_107_1', '108_109_1', '17_113_1', '114_115_1', '76_118_1', '30_17_1', '64_61_1']
Number of executed Power Flows: 1
losses: 496.7852924832344
radia

In [24]:
#Executes the DNR algorithm by using the Khalil method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, method='Khalil', NumCandidates=10, fitness_ratio=1, loss_factor=0.02)
print("118 buses case can not be solved with our Khalil implementation, as it does not manage to create the initial population")
#executeTest(dnr118, method='Khalil', NumCandidates=10, fitness_ratio=1, loss_factor=0.02)

33 buses case
DNR solution by  Khalil : ['9_10_1', '17_18_1', '27_28_1', '4_5_1', '13_14_1']
Number of executed Power Flows: 240
losses: 0.17346076362266663
radiality: (True, True, True)
Loops:
118 buses case can not be solved with our Khalil implementation, as it does not manage to create the initial population


In [25]:
#Executes the DNR algorithm by using the Jakus method to find the optimal configuration of the distribution network
print("33 buses case")
executeTest(dnr33, 'Jakus', TieLinesID=TieLinesID33, MutationProbability=0.4, PopulationSize=16, Niter=20, ElitePopulation=2, fitness_ratio=1, loss_factor=0.02)
print("118 buses case")
executeTest(dnr118, 'Jakus', TieLinesID=TieLinesID118, MutationProbability=0.4, PopulationSize=16, Niter=20, ElitePopulation=2, fitness_ratio=1, loss_factor=0.02)

33 buses case
DNR solution by  Jakus : ['7_8_1', '10_11_1', '9_15_1', '18_33_1', '25_29_1']
Number of executed Power Flows: 298
losses: 0.14510836742474806
radiality: (True, True, True)
Loops:
118 buses case
DNR solution by  Jakus : ['1_2_1', '3_5_1', '4_11_1', '7_12_1', '13_15_1', '12_16_1', '15_19_1', '22_23_1', '28_29_1', '8_30_1', '23_32_1', '27_32_1', '15_33_1', '19_34_1', '35_36_1', '37_40_1', '30_38_1', '40_41_1', '40_42_1', '44_45_1', '46_47_1', '46_48_1', '47_49_1', '52_53_1', '54_55_1', '54_56_1', '56_57_1', '56_58_1', '55_59_1', '59_60_1', '60_61_1', '61_62_1', '63_64_1', '64_65_1', '62_66_1', '49_69_1', '24_70_1', '70_75_1', '74_75_1', '75_77_1', '78_79_1', '68_81_1', '84_85_1', '85_88_1', '90_91_1', '92_93_1', '94_95_1', '80_96_1', '82_96_1', '94_96_1', '94_100_1', '98_100_1', '99_100_1', '100_101_1', '103_104_1', '104_105_1', '105_106_1', '105_107_1', '108_109_1', '17_113_1', '114_115_1', '76_118_1']
Number of executed Power Flows: 298
losses: 268.1643316042296
radiality:

In [26]:
## this method takes several hours to execute for the 33 buses case, finding more than 100K possible networks to be evaluated
## the optimal radial network has the following lines disabled : 
## obtaining losses of 139 kW

#The functions are commented to avoid its execution
#executeTest(dnr33, 'Morton', TieLinesID=TieLinesID33)
#executeTest(dnr118, 'Morton', TieLinesID=TieLinesID33)


In [ ]:
#the Taylor method is failing and needs to be reviewed

In [31]:
## implementation of Taylor Mathematical Programming method is not working properly and needs to be completely revised
disabled_lines = dnr33.Solve(method="Taylor", algorithm="QP", solver="ipopt", bigM=10e6, Imax=10, vmin=0.8, vmax=1.2, TieLines=TieLinesID33)
_, loss = GC_utils.GC_PowerFlow(dnr33.grid, config=disabled_lines)
radiality = GC_utils.CheckRadialConnectedNetwork(dnr33.grid)
print(f"Taylor: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr33.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )

#Taylor method 
disabled_lines = dnr118.Solve(method="Taylor", algorithm="QP", solver="ipopt", bigM=10e6, Imax=10, vmin=0.8, vmax=1.2, TieLines=TieLinesID118)
_, loss = GC_utils.GC_PowerFlow(dnr118.grid, config=disabled_lines)
radiality = GC_utils.CheckRadialConnectedNetwork(dnr118.grid)
print(f"Taylor: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr118.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )


2025-07-30 14:58:25 - pyomo.core - ERROR - Rule failed when generating expression for Constraint Constraint_5_Distflow with index 87:
ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (False) instead of a Pyomo object. Please modify your rule to return Constraint.Infeasible instead of False.

Error thrown for Constraint 'Constraint_5_Distflow['87']'
2025-07-30 14:58:25 - pyomo.core - ERROR - Constructing component 'Constraint_5_Distflow' from data=None failed:
    ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (False) instead of a Pyomo object. Please modify your rule to return Constraint.Infeasible instead of False.

Error thrown for Constraint 'Constraint_5_Distflow['87']'


Taylor: The new optimal configuration losses:1.8328618489172303e-05, radiality:(False, True, False), numPF:298 


ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (False) instead of a Pyomo object. Please modify your rule to return Constraint.Infeasible instead of False.

Error thrown for Constraint 'Constraint_5_Distflow['87']'

In [32]:
## implementation of Taylor Mathematical Programming method is not working properly and needs to be completely revised
disabled_lines = dnr33.Solve(method="Taylor", algorithm="SOC", solver="ipopt", bigM=10e6, Imax=10, vmin=0.8, vmax=1.2, TieLines=TieLinesID33)
_, loss = GC_utils.GC_PowerFlow(dnr33.grid, config=disabled_lines)
radiality = GC_utils.CheckRadialConnectedNetwork(dnr33.grid)
print(f"Taylor: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr33.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )

#Taylor method 
disabled_lines = dnr118.Solve(method="Taylor", algorithm="SOC", solver="ipopt", bigM=10e6, Imax=10, vmin=0.8, vmax=1.2, TieLines=TieLinesID118)
_, loss = GC_utils.GC_PowerFlow(dnr118.grid, config=disabled_lines)
radiality = GC_utils.CheckRadialConnectedNetwork(dnr118.grid)
print(f"Taylor: The new optimal configuration losses:{loss}, radiality:{radiality}, numPF:{dnr118.NumPF} ") #is {GC_utils.GC_Line_idtag2name_array(gridGC, disabled_lines)}" )


2025-07-30 14:58:30 - pyomo.core - ERROR - Rule failed when generating expression for Constraint Constraint_21_SOC with index 1:
KeyError: "Index '('2', '1')' is not valid for indexed component 'P_ij'"
2025-07-30 14:58:30 - pyomo.core - ERROR - Constructing component 'Constraint_21_SOC' from data=None failed:
    KeyError: "Index '('2', '1')' is not valid for indexed component 'P_ij'"


KeyError: "Index '('2', '1')' is not valid for indexed component 'P_ij'"